In [1]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from datasets import Dataset, load_dataset

import numpy as np
import pandas as pd
import evaluate
import accelerate
from transformers import AutoTokenizer, pipeline
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoConfig

c:\Users\ivanc\OneDrive - Singapore University of Technology and Design\School Work\Term_8\AI\Project\Depression_detection_project\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Old read csvs

In [ ]:
dev_df = pd.read_csv('data/dev.tsv', delimiter='\t')

In [53]:
print(len(dev_df))
print(len(dev_df[dev_df['label'] == 'moderate']))
print(len(dev_df[dev_df['label'] == 'not depression']))

4496
2306
1830


In [54]:
dev_df

,Unnamed: 0,PID,text,label
0,0,dev_pid_1,"I enjoyed today, and I still am! Tomorrows dep...",moderate
1,1,dev_pid_2,I sorta tried to kill myself : I had a total b...,moderate
2,2,dev_pid_3,Best suicide method? : I like it quick and eas...,moderate
3,3,dev_pid_4,a story : I remember the time I'd get on my 3D...,moderate
4,4,dev_pid_5,The world only cares about beautiful people : ...,moderate
...,...,...,...,...
4491,4491,dev_pid_4492,Aren’t we all just tired? : I’ve been depresse...,severe
4492,4492,dev_pid_4493,NEED HELP COPING : I had my life pretty much f...,severe
4493,4493,dev_pid_4494,Qutting Zoloft Cold Turkey : I was on 75 mg se...,severe
4494,4494,dev_pid_4495,Crying : I’m coming off my antidepressants and...,severe


In [56]:
# train_df = pd.read_csv('data/train_aug.tsv', delimiter='\t')
# train_df = pd.read_csv('data/balanced_train.tsv', delimiter='\t')
train_df = pd.read_csv('./data/train.tsv', delimiter='\t')

In [57]:
print(len(train_df))
print(len(train_df[train_df['label'] == 'moderate']))
print(len(train_df[train_df['label'] == 'not depression']))

8891
6004
1985


In [58]:
train_df

,Unnamed: 0,PID,text,label
0,0,train_pid_1,Waiting for my mind to have a breakdown once t...,moderate
1,1,train_pid_2,My new years resolution : I'm gonna get my ass...,moderate
2,2,train_pid_3,New year : Somone else Feeling like 2020 will ...,moderate
3,3,train_pid_4,"My story I guess : Hi, Im from Germany and my ...",moderate
4,4,train_pid_5,Sat in the dark and cried myself going into th...,moderate
...,...,...,...,...
8886,8886,train_pid_8887,Ways to reverse memory loss from depression? :...,severe
8887,8887,train_pid_8888,A Comprehensive Guide To Slowly Getting Better...,severe
8888,8888,train_pid_8889,I don’t think college is right for me : TW: su...,severe
8889,8889,train_pid_8890,Please help: Severe insomnia affecting me in m...,severe


In [60]:
test_df = pd.read_csv('data/test.tsv', delimiter='\t')

In [61]:
test_df

,Unnamed: 0,PID,text,label
0,0,test_pid_1,Im scared : This is it. I lie to myself every ...,moderate
1,1,test_pid_2,New to this but just wanted to vent : I just f...,moderate
2,2,test_pid_3,I’m sad : It’s kinda always been an issue. I w...,moderate
3,3,test_pid_4,Lonely but not alone. : All of my immediately ...,moderate
4,4,test_pid_5,This year has been trash. : I dont know why I’...,moderate
...,...,...,...,...
3240,3240,test_pid_3241,"Feeling lonely. : Hi reddit, I haven’t posted ...",severe
3241,3241,test_pid_3242,When would suicide be right? : So I got back f...,severe
3242,3242,test_pid_3243,Lowest I’ve ever been ever. : To make a long s...,severe
3243,3243,test_pid_3244,Does the Toxoplasma Gondii ruined my life ? (f...,severe


In [62]:
train_df = pd.read_csv("./data/reshuffle_train.tsv", delimiter='\t')
dev_df = pd.read_csv("./data/reshuffle_dev.tsv", delimiter='\t')
test_df = pd.read_csv("./data/reshuffle_test.tsv", delimiter='\t')

Preprocess the datasets

Firstly, we have to convert the labels from not depression, moderate, severe to 0, 1, 2

Then we have to tokenize and truncate the text

In [64]:
model_name = 'roberta-base'
rob_tokenizer = AutoTokenizer.from_pretrained(model_name)
max_len = 512

def process_labels(example):
    new_label = {'not depression': 0, 'moderate': 1, 'severe': 2}.get(example['label'], example['label'])
    return {'label': new_label}
    
def process_text(example, tokenizer=rob_tokenizer, max_len=512, padding='max_length', truncation=True):
    return tokenizer(example['text'], max_length=max_len, padding=padding, truncation=truncation)

In [65]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
dev_dataset = Dataset.from_pandas(dev_df)

In [66]:
train_dataset = train_dataset.map(process_labels)
train_dataset = train_dataset.map(process_text, batched=True)

test_dataset = test_dataset.map(process_labels)
test_dataset = test_dataset.map(process_text, batched=True)

dev_dataset = dev_dataset.map(process_labels)
dev_dataset = dev_dataset.map(process_text, batched=True)

Map: 100%|██████████| 2496/2496 [00:00<00:00, 5556.08 examples/s]


Train model using AdamW

different LR

different dropout

In [67]:
## Code for calculating weights for weighted binary prediction

from collections import Counter
import torch

# Example: assuming train_dataset has a 'label' field
labels = [example['label'] for example in train_dataset]
class_counts = Counter(labels)

# Compute positive class weight for BCEWithLogitsLoss
# pos_weight = # of non-depression / # of depression
pos_weight = torch.tensor([class_counts[0] / class_counts[1]])


In [68]:
from transformers import Trainer
from torch.nn import BCEWithLogitsLoss

class WeightedLossTrainer(Trainer):
    def __init__(self, pos_weight, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.pos_weight = pos_weight

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Compute weighted loss
        loss_fct = BCEWithLogitsLoss(pos_weight=self.pos_weight.to(logits.device))
        loss = loss_fct(logits.view(-1), labels.float().view(-1))

        return (loss, outputs) if return_outputs else loss


In [69]:
NUM_LABELS = 3 ## 0 or 1 for no depression and depression
MODEL_LABEL = "labels3_reshuffled_trainDevTest"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_LABELS)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [70]:
accuracy = evaluate.load("accuracy")
f1_score = evaluate.load('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"],
        "f1": f1_score.compute(predictions=predictions, references=labels, average="macro")["f1"]
    }

In [71]:
training_args = TrainingArguments(
    output_dir=f"./results/{MODEL_LABEL}",
    evaluation_strategy="epoch",  # evaluates on dev after each epoch
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True, # <- Load best model based on F1
    metric_for_best_model="f1", # <- Use F1 as selection metric
    greater_is_better=True, # <- Higher F1 is better
    fp16=True
)


c:\Users\ivanc\OneDrive - Singapore University of Technology and Design\School Work\Term_8\AI\HW\env\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [72]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
    # pos_weight=pos_weight # for weighted loss trainer
)


C:\Users\ivanc\AppData\Local\Temp\ipykernel_32268\714501680.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [73]:
train_log = trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.672900,0.641393,0.739984,0.643283
2,0.503900,0.570789,0.762821,0.701639
3,0.382700,0.781346,0.765224,0.706025
4,0.326800,0.937771,0.772436,0.716803
5,0.240000,1.253358,0.776843,0.729033
6,0.177600,1.427183,0.772837,0.717857
7,0.139700,1.577010,0.768429,0.714701
8,0.091800,1.716262,0.772035,0.718817


In [11]:
from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import load_dataset
import evaluate
import numpy as np
import torch
import os

NUM_LABELS = 2
MODEL_TYPE = 'Binary_Classification'

# Assuming tokenizer, train_dataset, dev_dataset are already defined and preprocessed
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

# Hyperparameter options
LRs = [2e-5, 5e-5, 1e-4, 3e-4]
weight_decays = [0.01, 0.02]

# Loop through all combinations
for lr in LRs:
    for wd in weight_decays:
        run_name = f"{MODEL_TYPE}lr_{lr}_wd_{wd}"
        output_dir = f"./results/{run_name}"

        print(f"\nStarting training for {run_name}")

        model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=NUM_LABELS)

        training_args = TrainingArguments(
            output_dir=output_dir,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=lr,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            num_train_epochs=10,
            weight_decay=wd,
            logging_dir=f"./logs/{run_name}",
            load_best_model_at_end=True,
            metric_for_best_model="accuracy",
            logging_steps=10,
            save_total_limit=1,  # only keep best checkpoint
            fp16=True
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=dev_dataset,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )

        trainer.train()

        # Save final model and metrics
        trainer.save_model(f"{output_dir}/final_model")
        metrics = trainer.evaluate()
        with open(f"{output_dir}/metrics.txt", "w") as f:
            for key, val in metrics.items():
                f.write(f"{key}: {val}\n")

        print(f"Finished {run_name} - Accuracy: {metrics.get('eval_accuracy', 'N/A')}")



Starting training for Binary_Classificationlr_2e-05_wd_0.01


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\ivanc\OneDrive - Singapore University of Technology and Design\School Work\Term_8\AI\HW\env\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\ivanc\AppData\Local\Temp\ipykernel_20692\3965535182.py:56: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.449700,0.830032,0.686610
2,0.335600,1.393730,0.652135
3,0.237400,1.055826,0.645685
4,0.188900,1.554594,0.670151
5,0.003800,2.054934,0.681495
6,0.001500,2.009965,0.671041
7,0.000200,2.401954,0.678381
8,0.022400,2.429269,0.676157
9,0.000500,2.544962,0.685943
10,0.029900,2.586025,0.684609


Finished Binary_Classificationlr_2e-05_wd_0.01 - Accuracy: 0.6866103202846975

Starting training for Binary_Classificationlr_2e-05_wd_0.02


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\ivanc\OneDrive - Singapore University of Technology and Design\School Work\Term_8\AI\HW\env\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\ivanc\AppData\Local\Temp\ipykernel_20692\3965535182.py:56: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.645400,0.768319,0.592972
2,0.574600,0.848548,0.592972
3,0.736700,0.753376,0.592972
4,0.472200,0.794095,0.592972
5,0.502200,0.815620,0.592972
6,0.542700,0.768490,0.592972
7,0.470300,0.835490,0.592972
8,0.597000,0.756986,0.592972
9,0.548400,0.760233,0.592972
10,0.534200,0.766180,0.592972


Finished Binary_Classificationlr_0.0003_wd_0.01 - Accuracy: 0.5929715302491103

Starting training for Binary_Classificationlr_0.0003_wd_0.02


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\ivanc\OneDrive - Singapore University of Technology and Design\School Work\Term_8\AI\HW\env\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\ivanc\AppData\Local\Temp\ipykernel_20692\3965535182.py:56: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.647500,0.772036,0.592972
2,0.566600,0.851762,0.592972
3,0.734500,0.754242,0.592972


KeyboardInterrupt: 